# Import libraries

In [ ]:
import sys
import glob, os
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
from plotly import tools

from scipy.spatial import distance
from scipy import linalg
from scipy import signal

%matplotlib inline
offline.init_notebook_mode()

# Load data

In [ ]:
path = '../../machine_learning/cloud_functions/data-large.csv'
data = pd.read_csv(path)
df = pd.DataFrame(data)
df.head()

In [ ]:
columns = ['temporal_canny-series',
           'temporal_cross_correlation-series',
           'temporal_dct-series',
           'temporal_difference-series',
           'temporal_histogram_distance-series',
           'temporal_gaussian_mse-series',
           'dimension',
           'attack',
           'title']

df = df[columns]
df.head()

# Create functions

In [ ]:
def select_time_series(df, series_name, asset_name=None):
    if not asset_name:
        asset_name = df.sample(1)['title'].values[0]
    else:
        asset_name = '/tmp/' + asset_name 
    print('{} was selected'.format(str(asset_name)))
    
    series = []
    asset_info = df[df['title'] == asset_name][[series_name, 'dimension', 'attack']]  
    for i, row in asset_info.iterrows():
        series.append(np.fromstring(row[series_name].replace('[', '').replace(']', ''), 
                                                    dtype=np.float, sep=' '))
    asset_info['series'] = series
    return asset_info


# Series analysis

In [ ]:
# Select series to study, if not specified a random video will be used

series = 'temporal_gaussian_mse-series'
df_series = select_time_series(df, series)
display(df_series.head())

In [ ]:

X = list(range(0, len(df_series['series'].values[0] -1) ))

data = []
scaling = False
i = 0
for _, row in df_series.iterrows():
    
    Y = df_series.iloc[i]['series']
    
    if scaling:
        Y /= df_series.iloc[i]['dimension']
    
    trace = go.Scatter(
            x = X,
            y = Y,
            name = df_series.iloc[i]['attack'],
            mode = 'lines'
        )
    data.append(trace)
    i += 1

layout = {'title': series, 
      'legend':{'x': 1, 'y':.95},
      'xaxis': {'title': 'Frame', }, 
      'yaxis': {'title': series},
      'hovermode':'closest'
      }
fig = go.Figure(data=data, layout=layout)
offline.iplot(fig)
